In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"qasmovdyant","key":"4d4759066dca189587bcac8981b34740"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sahideseker/spam-mail-classifier-dataset

Dataset URL: https://www.kaggle.com/datasets/sahideseker/spam-mail-classifier-dataset
License(s): CC-BY-SA-4.0


In [ ]:
!wget https://raw.githubusercontent.com/databyhuseyn/DeepLearning/main/helper_functions.py

--2025-04-30 07:59:21--  https://raw.githubusercontent.com/databyhuseyn/DeepLearning/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-04-30 07:59:22 (51.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import unzip_data

In [ ]:
unzip_data('/content/spam-mail-classifier-dataset.zip')

In [ ]:
data = pd.read_csv('/content/spam_mail_classifier.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   email_text  1000 non-null   object
 1   label       1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [ ]:
data.describe()

,email_text,label
count,1000,1000
unique,10,2
top,Let's catch up sometime next week!,ham
freq,129,583


In [ ]:
df_shuffled = data.sample(frac=1, random_state=42)
df_shuffled

,email_text,label
521,Don't forget to submit your project by Friday.,ham
737,Let's catch up sometime next week!,ham
740,"Hey, are we still on for lunch tomorrow?",ham
660,Let's catch up sometime next week!,ham
411,Congratulations! You’ve been selected for a ca...,spam
...,...,...
106,Don't forget to submit your project by Friday.,ham
270,Meeting has been rescheduled to next Monday.,ham
860,Let's catch up sometime next week!,ham
435,You won a lottery. Claim your money now.,spam


In [ ]:
X = df_shuffled['email_text']
y = df_shuffled['label']

In [ ]:
y.value_counts()

,count
label,
ham,583
spam,417


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(sparse_output=False)

y_train = onehot.fit_transform(y_train.values.reshape(-1, 1))
y_test = onehot.transform(y_test.values.reshape(-1, 1))

In [ ]:
X_test.head()

,email_text
927,Can you send me the report when it's ready?
56,Don't forget to submit your project by Friday.
426,Meeting has been rescheduled to next Monday.
295,Congratulations! You’ve been selected for a ca...
122,Congratulations! You’ve been selected for a ca...


In [ ]:
max_length = round(sum([len(i.split()) for i in X_train]) / (len(X_train)))

In [ ]:
max_length

7

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

max_vocab_length = 10_000
text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length = max_length)

In [ ]:
text_vectorizer.adapt(X_train)

In [ ]:
sample_sentence = "E-poçt hesabınız 3,700,000,00 funt sterlinq ianə üçün seçilmişdir."
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 7), dtype=int64, numpy=array([[1, 1, 1, 1, 1, 1, 1]])>

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             embeddings_initializer = 'uniform',
                             input_length = max_length,
                             name = 'embedding')

In [ ]:
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, activation='relu', kernel_regularizer = 'l2')(x)
outputs = layers.Dense(len(data['label'].value_counts()),activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs, name = 'model_0')

In [ ]:
model.compile(loss = 'categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_0"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization              │ (None, 7)              │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 7, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,538 (5.07 MB)

 Trainable params: 1,329,538 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_history = model_0.fit(X_train.to_numpy(),
                              y_train,
                              epochs=5,
                              validation_data = (X_test.to_numpy(), y_test))

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 1.0000 - loss: 3.6116e-04 - val_accuracy: 1.0000 - val_loss: 2.3919e-04
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 1.0000 - loss: 2.1674e-04 - val_accuracy: 1.0000 - val_loss: 1.6597e-04
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 1.0000 - loss: 1.5532e-04 - val_accuracy: 1.0000 - val_loss: 1.2942e-04
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 1.0000 - loss: 1.2314e-04 - val_accuracy: 1.0000 - val_loss: 1.0686e-04
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 1.0000 - loss: 1.0247e-04 - val_accuracy: 1.0000 - val_loss: 9.0662e-05


In [ ]:
prediction_position = np.argmax(model.predict(tf.constant(['That “aha!” moment when everything clicks?'])))
prediction_position

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step


np.int64(0)

In [ ]:
models_prediction = onehot.get_feature_names_out()[prediction_position][3:]
models_prediction

'ham'